In [1]:
import os, sys

sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath("__file__"))))
from nbafuns import *

fig_DIR = "../figs/teams/"
box_DIR = "../data/box/"

In [2]:
from nba_api.stats.endpoints import leaguedashteamstats

In [3]:
for season in range(1996,2025):
    stats = leaguedashteamstats.LeagueDashTeamStats(
        measure_type_detailed_defense="Base",
        season=get_ss(season),
    )
    df = stats.get_data_frames()[0]
    df.to_parquet(box_DIR + f"NBA_Box_T_Cum_Base_{season}.parquet")
    time.sleep(1)

In [79]:
dfa = []
for season in range(2014,2025):
    df1 = pd.read_parquet(box_DIR + f"NBA_Box_T_Cum_Adv_{season}.parquet")
    df2 = df1[["TEAM_NAME","NET_RATING"]]
    df2["Season"] = season +1
    dfa.append(df2)
df3 = pd.concat(dfa)

In [80]:
dfg = df3.groupby("TEAM_NAME")
teams = list(dfg.groups)
dfb = []
for t in teams:
    df4 = dfg.get_group(t)
    df4["NET_RATING_Prev"] = df4["NET_RATING"].shift(1)
    df4["NET_RATING_Diff"] = df4["NET_RATING"]-df4["NET_RATING_Prev"]
    df4 = df4.dropna()
    dfb.append(df4)
df5 = pd.concat(dfb)

In [82]:
df6 = df5.sort_values("NET_RATING_Diff",ascending=False)
df6 = df6.reset_index(drop=True)
df6 = df6[["TEAM_NAME",	"Season",	"NET_RATING_Prev",	"NET_RATING",	"NET_RATING_Diff",]]
df6 = df6.rename(columns={"TEAM_NAME":"Team",	"NET_RATING_Prev":"Prev",	"NET_RATING":"Net Rating",	"NET_RATING_Diff":"Diff"})
df6.index +=1
df6.head(10)

,Team,Season,Prev,Net Rating,Diff
1,Memphis Grizzlies,2025,-6.9,7.0,13.9
2,Cleveland Cavaliers,2022,-8.3,2.1,10.4
3,Philadelphia 76ers,2018,-5.5,4.6,10.1
4,Golden State Warriors,2021,-8.6,1.1,9.7
5,Atlanta Hawks,2021,-7.4,2.2,9.6
6,Phoenix Suns,2020,-8.9,0.5,9.4
7,Cleveland Cavaliers,2025,2.5,11.8,9.3
8,Oklahoma City Thunder,2023,-8.0,1.0,9.0
9,Houston Rockets,2024,-8.1,0.9,9.0
10,New York Knicks,2021,-6.5,2.4,8.9
